In [1]:
import os

In [2]:
%pwd

'/Users/c5367983/Desktop/Projects/QuantileX/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/Users/c5367983/Desktop/Projects/QuantileX'

In [5]:
from src.constants import *
from src.utils.common import read_yaml, create_directories

In [6]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    data_path: Path
    output_path: Path
    assets_type: str
    alpha: float
    


In [7]:
from sklearn.model_selection import TimeSeriesSplit
from sklearn.linear_model import Lasso, Ridge
from joblib import dump, load
from sklearn.metrics import mean_absolute_error  
from sklearn.metrics import mean_squared_error   
from sklearn.metrics import r2_score             
import pandas as pd
import numpy as np

In [8]:
class ConfigurationManager:
    def __init__(
        self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.TrainingArguments

        create_directories([config.root_dir + "/" + config.assets_type])
        create_directories([config.output_path])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            output_path=config.output_path,
            assets_type=config.assets_type,
            alpha=params.alpha
        )
        return model_trainer_config
    


In [9]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config
        self.data_frame = pd.read_csv(f"{self.config.data_path}/{self.config.assets_type}.csv")
        self.data_frame['Date'] = pd.to_datetime(self.data_frame['Date'])
        self.data_frame['Close'] = self.data_frame['Close'].shift(-1)
        self.data_frame = self.data_frame.set_index('Date')
        self.last_row = self.data_frame.tail(1).drop("Close",axis=1)
        self.data_frame.drop(self.data_frame.index[-1],inplace = True)
        self.y = self.data_frame['Close']
        self.X = self.data_frame.drop('Close', axis = 1)
        self.metrics = {"MSE":[], "MAE":[], "r2":[]}

    def train(self):
        tscv = TimeSeriesSplit(n_splits=5)
        lasso_model = Lasso(alpha=0.1)
        for train_index, test_index in tscv.split(self.X):
            #print(f"Train index: {train_index}")
            X_train, X_test = self.X.iloc[train_index], self.X.iloc[test_index]
            y_train, y_test = self.y.iloc[train_index], self.y.iloc[test_index]
            
            lasso_model.fit(X_train, y_train)
            predictions = lasso_model.predict(X_test)
            mse = mean_squared_error(y_test, predictions)
            mae = mean_absolute_error(y_test, predictions)
            r2 = r2_score(y_test, predictions)
            print(f"MSE: {mse:.4f}, MAE: {mae:.4f}, R-squared: {r2:.4f}")
            self.metrics["MSE"].append(mse)
            self.metrics["MAE"].append(mae)
            self.metrics["r2"].append(r2)
        dump(lasso_model, f"{self.config.root_dir}/{self.config.assets_type}/Lasso_for_close.joblib")

    def predict(self):
        loaded_model = load(f"{self.config.root_dir}/{self.config.assets_type}/Lasso_for_close.joblib")
        close_prediction = loaded_model.predict(self.last_row)
        close_low = float(close_prediction[0]) - float(self.metrics["MAE"][-1])
        close_high = float(close_prediction[0]) + float(self.metrics["MAE"][-1])
        final_output = {"Close_prediction":close_prediction[0], "Close_low":close_low, "Close_high":close_high}
        df_output = pd.DataFrame([final_output])
        df_output.to_json(f"{self.config.output_path}/output_of_{self.config.assets_type}.json", orient='records', lines=True, indent=4)
    

In [10]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
    model_trainer_config.predict()
except Exception as e:
    raise e

[2024-02-25 22:36:16,012: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-02-25 22:36:16,014: INFO: common: yaml file: params.yaml loaded successfully]
[2024-02-25 22:36:16,014: INFO: common: Created directory at: artifacts]
[2024-02-25 22:36:16,014: INFO: common: Created directory at: artifacts/model_trainer/BTC-USD]
[2024-02-25 22:36:16,015: INFO: common: Created directory at: artifacts/output]
MSE: 44893384.8340, MAE: 1906.9278, R-squared: -1.5253


/Users/c5367983/anaconda3/envs/quantilex/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.679e+04, tolerance: 8.664e+02
  model = cd_fast.enet_coordinate_descent(
/Users/c5367983/anaconda3/envs/quantilex/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.951e+07, tolerance: 1.248e+06
  model = cd_fast.enet_coordinate_descent(


MSE: 1705359.1357, MAE: 1206.7196, R-squared: 0.6770


/Users/c5367983/anaconda3/envs/quantilex/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.353e+07, tolerance: 2.464e+06
  model = cd_fast.enet_coordinate_descent(


MSE: 491635.6004, MAE: 442.1510, R-squared: 0.9907


/Users/c5367983/anaconda3/envs/quantilex/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.348e+08, tolerance: 8.533e+06
  model = cd_fast.enet_coordinate_descent(


MSE: 2930769.4595, MAE: 1257.3033, R-squared: 0.9788


/Users/c5367983/anaconda3/envs/quantilex/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.008e+08, tolerance: 7.710e+07
  model = cd_fast.enet_coordinate_descent(


MSE: 356895.5160, MAE: 398.8859, R-squared: 0.9953


In [11]:
obj = pd.read_json("artifacts/output/output_of_BTC-USD.json", lines=True)

ValueError: If using all scalar values, you must pass an index